### Итоговое задание Макаровой Елены по Юниту 8. "Нейронные сети" по Проекту 7. Ford vs Ferrari: определяем модель авто по фото

## **нужно** построить модель классификации изображений. Классифицировать мы будем автомобили по их фотографиям.  Применим навыки, которые получили в этом юните, и обучим такую модель!
## **Основная идея этого решения:** взять предобученую на ImageNet сеть EfficientNet и дообучить под нашу задачу.

# 1.
## 1.1. Импорты библиотек

In [ ]:
# проверяем, что видеокарта подключена
!nvidia-smi

In [ ]:
import os, re, math, time, gc,sys,csv, string, pickle,zipfile, shutil, pathlib, itertools
import numpy as np, pandas as pd, matplotlib.pyplot as plt, PIL, seaborn as sns, random as rn
from datetime import datetime as dt

import tensorflow as tf
import cv2
from tensorflow import keras
from tensorflow.keras import *
from tensorflow.keras.activations import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from IPython.display import clear_output
from tqdm.notebook import tqdm

from tensorflow.keras import optimizers
from tensorflow.keras import Sequential as S
import tensorflow.keras.models as M
from tensorflow.keras.layers import *
import tensorflow.keras.layers as L
from tensorflow.keras.regularizers import l2

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB3,EfficientNetB5
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.callbacks import Callback

from sklearn.model_selection import train_test_split, StratifiedKFold
from keras import backend as K

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

Работать будем с Tensorflow 2.

In [ ]:
# фиксируем версии всех пакетов для воспроизводимости результата
!pip freeze > requirements.txt
# фиксируем текущую дату для контроля версий экспериментов
CURRENT_DATE = dt.now().strftime('[%d.%m.%Y]') 

## 1.2. Настройка глобальных констант

In [ ]:
# выносим основные настройки, чтобы было удобнее их перебирать в дальнейшем:

EPOCHS               = 12  # эпох на обучение
BATCH_SIZE           = 32 # уменьшаем batch если сеть большая, иначе не поместится в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 320 # какого размера подаем изображения в сеть (224)
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

# Устаналиваем конкретное значение random seed для воспроизводимости результата
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
os.environ['PYTHONHASHSEED'] = '0'
rn.seed(RANDOM_SEED)

In [ ]:
CURRENT_DIR = '../'  # имя текущей директории для каггл
# проверка пути в папке input для корректного назначения глобальной константы 
print(os.listdir(CURRENT_DIR+'input/'))

# имя директории с исходными файлами
DATA_PATH = '../input/'
PATH_to_FILE = '../input/sf-dl-car-classification/'

# имя рабочей директории
PATH_to_WORKDIR = "../working/car/"  # PATH = "../working/car/" # рабочая директория
MODEL_PATH = '/kaggle/working'

TRAIN_PATH = '/kaggle/train/'
SUB_PATH = '/kaggle/test_upload/'

# PATH_to_FILE_RESULT = '../input/last-result/last_result.csv'
# PATH_to_FILE_descr = '../input/last-result/descr_result.pkl'


# 2. Импорт и доработка данных
## 2.1. Импорт данных

In [ ]:
train_df = pd.read_csv(PATH_to_FILE + 'train.csv')
df_submis = pd.read_csv(PATH_to_FILE + 'sample-submission.csv')

## 2.2. Доработка данных

In [ ]:
print('Распаковываем картинки')

# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    print(data_zip)
    with zipfile.ZipFile(f"{PATH_to_FILE}{data_zip}","r") as z:
        z.extractall(PATH_to_WORKDIR)
        
print(os.listdir(PATH_to_WORKDIR))

# 3. Разведовательный анализ данных (EDA)

### 3.1. Первоначальный анализ

In [ ]:
train_df.head(), df_submis.head()

In [ ]:
train_df.info(), df_submis.info()

Нет пропусков.

In [ ]:
print('Представленные категории:')
train_df.Category.unique()

In [ ]:
train_df.Category.value_counts()

В датасете представлено 10 марок машин

In [ ]:
fig, ax = plt.subplots()
sns.barplot(y=train_df.Category.value_counts().values,
           x=train_df.Category.value_counts().index,saturation=10,
           palette="Spectral")
ax.set_ylabel('Количество картинок в категории авто',fontsize=10)
ax.set_xlabel('Категория',fontsize=10)
ax.set_title('Распределение количества картинок по категориям авто',fontsize=11.5)

# распределение классов достаточно равномерное - это хорошо

### 3.2. Визуальный анализ картинок

Посмотрим на примеры картинок и их размеры, чтобы понимать, как их лучше обрабатывать и сжимать.

In [ ]:
print('Пример картинок')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH_to_WORKDIR+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Картинки представлены разного размера

In [ ]:
#Посмотрим на примеры картинок и их размеры, чтобы понимать, как их лучше обрабатывать и сжимать.
image = PIL.Image.open(PATH_to_WORKDIR+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

**Предварительные итоги:**
* Для классификации представлены фотографии 10 категорий авто VAZ, Volkswagen, Ford   
   1. 0 категория = Lada Priora
   2. 1 категория = Ford Focus
   3. 2 категория = Lada Samara 2114
   4. 3 категория = Lada 2110
   5. 4 категория = Lada 2107
   6. 5 категория - Niva
   7. 6 категория - Lada Kalina
   8. 7 категория - Lada 2109
   9. 8 категория - Volkswagen Passat
   10. 9 категория - Lada Samara 21099
* Категории достаточно сбалансированны по кол-ву фото
* Размеры фотографий различаются, но в основном 640 на 480. Цветные.
* Всего 22236 фото в том числе: 15561 в трейне и 6675 в тесте
* Кол-во фото в трейне не достаточно для хорошего обучения модели нейронной сети, поэтому будем применять различные виды аугментации данных

# 4.

## 4.1 Аугментация данных

In [ ]:
! pip install git+https://github.com/mjkvaak/ImageDataAugmentor
from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A

try:
    from ImageDataAugmentor.image_data_augmentor import *
except:
    import sys
    sys.path.append("..")
    from ImageDataAugmentor.image_data_augmentor import *

In [ ]:
# Аугментация данных важна, когда мы работаем с небольшим датасетом. Это как раз наш случай!
# Чтобы лучше понять работу параметров, можно попробовать их поизменять.
# Официальная документация: https://keras.io/preprocessing/image/
AUGMENTATIONS = A.Compose([
    A.GaussianBlur(p=0.05), # размытие
    A.RandomBrightness(limit=0.2, p=0.6),#изменение яркости входного изображения
    A.ShiftScaleRotate(shift_limit=(-0.06, 0.12), scale_limit=(-0.02,0.14), interpolation=1, 
                       border_mode=4, rotate_limit=(-76,74), p=0.7),# произвольные преобразования: поворот ,масштаб,перевод
    A.OneOf([A.CenterCrop(height=224, width=200),A.CenterCrop(height=200, width=224)],p=0.5),# обрезка
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=(-0.15, 0.3), contrast_limit=0.2),#Произвольное изменение яркости и контрастности входного изображения
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)], p=0.5),
    A.HorizontalFlip(p=1), # отражение по горизонтале
    A.HueSaturationValue(p=0.5,hue_shift_limit=(-10, 50),sat_shift_limit=(-15, 15),val_shift_limit=(-10,5)), #Произвольное изменение оттенка, насыщенности и значения входного изображения.
    A.RGBShift(p=0.5),
    A.FancyPCA(alpha=1.3, always_apply=False, p=0.5),
    A.Resize(224, 320) #Измените размер ввода на заданную высоту и ширину
])

In [ ]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        validation_split=VAL_SPLIT,
        )

test_datagen = ImageDataGenerator(rescale=1. / 255)

## 4.2 Генерация данных

In [ ]:
# Завернем наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    PATH_to_WORKDIR+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH_to_WORKDIR+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=df_submis,
    directory=PATH_to_WORKDIR+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)
# Обратите внимание, что для сабмита мы используем другой источник test_datagen.flow_from_dataframe.

Посмотрим как изменились картинки

In [ ]:
#  Глянем на аугментированные каритнки train

from skimage import io
def imshow(image_RGB):
    io.imshow(image_RGB)
    io.show()

x,y = train_generator.next() # вызываем трейн-генератор
print('Пример картинок из train_generator')
plt.figure(figsize=(12,8))

for i in range(0,6):
    image = x[i]
    plt.subplot(3,3, i+1)
    plt.imshow(image)
    #plt.title('Class: '+str(y[i]))
    #plt.axis('off')
plt.show()

In [ ]:
#  Глянем на аугментированные каритнки test
x,y = test_generator.next() # вызываем тест-генератор
print('Пример картинок из test_generator')
plt.figure(figsize=(12,8))

for i in range(0,6):
    image = x[i]
    plt.subplot(3,3, i+1)
    plt.imshow(image)
    #plt.title('Class: '+str(y[i]))
    #plt.axis('off')
plt.show()

## Обратный вызов Callbackss

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
MODEL = "efficient-b6"
MODEL_PATH = '/kaggle/working'
def make_callbacks():
    
    callback_early_stopping = EarlyStopping(monitor='accuracy',patience=2, verbose=1)
    callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.5,min_lr=1e-10,patience=0,verbose=1)
    callback_learing_rate = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x),
    path_checkpoint = 'checkpoint.keras'
    callback_checkpoint = ModelCheckpoint(f'{MODEL}_best.hdf6' , monitor = ['val_acc'] , verbose = 1  , mode = 'max') 
    #ModelCheckpoint(filepath=path_checkpoint,monitor='val_loss',verbose=1,save_weights_only=True, save_best_only=True)

    return [callback_checkpoint,
            callback_learing_rate,
                 callback_reduce_lr]

callbacks = make_callbacks()

# Построение модели

In [ ]:
input_shape

В keras есть дополнительная функция tf.keras.applications, которая позволяет подгрузить уже готовую модель с весами. Если посмотреть в документации функции, там доступны различные модели.  Но мы воспользуемся EfficientNetB6 - у нее достаточно приемлемое качество.

Подгрузим веса (weights='imagenet') предобученные на имадженете

не будем использовать выходы, которые использовались на имедженете(include_top=False)

и указываем размер картинок input_shape=input_shape 

In [ ]:
#base_model = Xception(weights='imagenet', 
#                      include_top=False, 
#                      input_shape = input_shape)

#base_model = InceptionV3(weights='imagenet', 
#                         include_top=False, 
#                         input_shape = input_shape)

# base_model = efn.EfficientNetB5(weights='imagenet', 
#                                 include_top=False, 
#                                 input_shape = input_shape)

# Загрузка модели efficientnet
!pip install -q efficientnet
import efficientnet.tfkeras as efn
# подгружаем предобученую сеть EfficientNetB6
base_model = efn.EfficientNetB6(weights='imagenet',include_top=False,input_shape = input_shape)


In [ ]:
base_model.summary()

подгрузились веса.
Смотрим саму модель.
Как видим, достаточно обьемная модель

In [ ]:
# Для начала заморозим веса EfficientNetB6 и обучим только "голову". 
# Делаем это для того, чтобы хорошо обученные признаки на Imagenet не затирались в самом начале нашего обучения
base_model.trainable = False

## Далее, так сказать, утанавливаем "свою голову".

In [ ]:
model = M.Sequential()
model.add(base_model) # указываем что соединяем base_model
model.add(L.GlobalAveragePooling2D(),)# делаем GAP который объединяет все признаки в единный вектор

# и далее устанавливаем  Dense слой,BatchNormalization,Dropout, и еще Dense слой.
model.add(L.Dense(256, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.25))
model.add(L.Dense(CLASS_NUM, activation='softmax'))

# впринципе, если сеть достаточно хороша, то достаточно одного денс-слоя. 

In [ ]:
#смотрим что за сеть в итоге.
model.summary()

In [ ]:
# сколько слоев
print(len(model.layers))

In [ ]:
# сколько слоев обучилось
len(model.trainable_variables)

# Обучение модели

### 1. Замораживаем базовую часть модели полностью

In [ ]:
callbacks = make_callbacks()

In [ ]:
#LR=0.0001 задали выше
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=LR), metrics=["accuracy"])

Так же воспользуемся функцией колбеков в кераси, это очень удобная вещь, она позваляет делать нам множество вещей во время обучения, одна из них - это можно делать чекпоинты (например,если у нас улучшился результат, то мы можем сохранить нашу модель, чтобы в дальнейшем подгрузить только самую успешную модель).

Так же у нас есть EarlyStopping - мы останавливаемся если наша модель уже не обучается дольше patience=5 интераций.

In [ ]:
# Добавим ModelCheckpoint. 
# Эта функция позволяет сохранять прогресс обучения модели, 
# чтобы в нужный момент можно было его подгрузить и дообучить модель.
checkpoint = ModelCheckpoint('best_model.hdf6', 
                             monitor = ['val_accuracy'], 
                             verbose = 1,
                             mode = 'max')
earlystop = EarlyStopping(monitor = 'val_accuracy',
                          patience = 5,
                          restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.25,
                              patience=2,
                              min_lr=0.0000001,
                              verbose=1,
                              mode='auto')
callbacks_list = [checkpoint, earlystop, reduce_lr]

In [ ]:
#проверили нашу модель по точности сейчас с рандомными весами. 10%
scores = model.evaluate(test_generator, steps=1, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

теперь обучим нашу модель

In [ ]:
# Обучаем
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list)

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf6')
model.load_weights('best_model.hdf6')

И проверяем точность

In [ ]:
scores = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

В Итоге точность нашей модели составила 71.2%. Учитывая что классов 10 - это неплохой результат!


Посмотрим графики обучения:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

### 2. Fine-tuning  Разморозка половины слоев базовой модели

суть дообучения (Fine Tuning) заключается в размораживании последних слоев нейронной сети (Neural Net) и их обучении. Таким образом, корректируются слои, которые имеют наиболее абстрактные представления. Производя дообучение только нескольких слоев, мы уменьшаем риск переобучения (overfitting). И самое главное, это позволяет сделать текущую модель ещё более подходящей к нашей задаче.

In [ ]:
# Давайте посмотрим, сколько слоев в базовой модели
len(base_model.layers)

In [ ]:
# начинаем постепенную разморозку слоев для базовой модели для обучения
base_model.trainable = True

fine_tune_at = len(base_model.layers)//2 # количество слоев, которые будут обучаться

# Заморозим все слои перед слоем `fine_tune_at`
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
len(base_model.trainable_variables)

In [ ]:
LR=0.00001
model.compile(loss="categorical_crossentropy",
              optimizer = optimizers.Adam(learning_rate = LR), 
              metrics = 'accuracy')

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_generator, 
    steps_per_epoch = len(train_generator), 
    validation_data = test_generator, 
    validation_steps = len(test_generator), 
    epochs = EPOCHS, 
    callbacks = callbacks_list
)

In [ ]:
model.save('../working/model_last.hdf6')
model.load_weights('best_model.hdf6')

In [ ]:
scores = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

### 3. Разморозим больше слоев

In [ ]:
base_model.trainable = True

fine_tune_at = len(base_model.layers)//4

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
len(base_model.trainable_variables) 

In [ ]:
LR = 1e-5
model.compile(loss="categorical_crossentropy", 
              optimizer = optimizers.Adam(learning_rate = LR), 
              metrics = 'accuracy')

In [ ]:
model.summary()

In [ ]:
# Обучаем
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = 20,
        callbacks = callbacks_list
)

In [ ]:
model.save('../working/model_last.hdf6')
model.load_weights('best_model.hdf6')

In [ ]:
scores = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

### 4. Разморозим полностью

In [ ]:
base_model.trainable = True
model.compile(loss="categorical_crossentropy",
              optimizer = optimizers.Adam(learning_rate = LR), 
              metrics = 'accuracy')

In [ ]:
history = model.fit(
    train_generator, 
    steps_per_epoch = len(train_generator), 
    validation_data = test_generator, 
    validation_steps = len(test_generator), 
    epochs = 12, 
    callbacks = callbacks_list
)

In [ ]:
scores = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model.save('../working/model_last.hdf6')
model.load_weights('best_model.hdf6')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## Submission

In [ ]:
test_sub_generator.samples

In [ ]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

In [ ]:
submission.head()

## 5. IMG_SIZE rise

In [ ]:
# изменим некоторые параметры
EPOCHS               = 8
BATCH_SIZE           = 8 
LR                   = 1e-5

IMG_SIZE             = 512
IMG_CHANNELS         = 3
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

In [ ]:
AUGMENTATIONS = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.07, 
                       scale_limit=0.01, 
                       interpolation=1, 
                       border_mode=4, 
                       rotate_limit=20, 
                       p=.75),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, interpolation=1, border_mode=4, 
             value=None, mask_value=None, always_apply=False, p=0.5),
])

In [ ]:
train_gen2 = ImageDataAugmentor(rescale=1./255,
                        augment=AUGMENTATIONS, 
                        seed=RANDOM_SEED,
                        validation_split=VAL_SPLIT
                       )
test_gen2 = ImageDataAugmentor(rescale=1./255)


# data generators
train_generator_2 = train_gen2.flow_from_directory(
        PATH_to_WORKDIR+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, 
        seed=RANDOM_SEED,
        subset='training')

test_generator_2 = test_gen2.flow_from_directory(
        PATH_to_WORKDIR+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, 
        seed=RANDOM_SEED,
        subset='validation')

test_sub_generator_2 = test_datagen.flow_from_dataframe( 
    dataframe=df_submis,
    directory=PATH_to_WORKDIR+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [ ]:
base_model = efn.EfficientNetB6(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
# Устанавливаем новую "голову" (head)

model = Model.Sequential()
model.add(base_model)

model.add(layers.GlobalAveragePooling2D())
model.add(layers.BatchNormalization())  

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.25))

model.add(layers.Dense(CLASS_NUM, activation='softmax'))

model.compile(loss="categorical_crossentropy", 
               optimizer=optimizers.Adam(learning_rate=LR), 
               metrics=["accuracy"],
               )

In [ ]:
# Загружаем веса уже обученной модели
model.load_weights('best_model.hdf6')

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=2, min_lr=0.0000001, verbose=1,
                             mode='auto')

callbacks_list = [checkpoint,earlystop,reduce_lr]

In [ ]:
history = model.fit(
        train_generator_2,
        steps_per_epoch = train_generator_2.samples//train_generator_2.batch_size,
        validation_data = test_generator_2, 
        validation_steps = test_generator_2.samples//test_generator_2.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
scores = model.evaluate(test_generator_2, steps=len(test_generator_2), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

## sabmission после увеличения размера картинки

In [ ]:
test_sub_generator_2.reset()
predictions = model.predict(test_sub_generator_2, steps=len(test_sub_generator_2), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_datagen.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator_2.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')

In [ ]:
submission.to_csv('submission_IMG_SIZE.csv', index=False)

## 6. Test Time Augmentation

https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

Аугментируем тестовые изображения и сделаем несколько предсказаний одной картинки в разном виде. Взяв среднее значение из нескольких предсказаний получим итоговое предсказание.

In [ ]:
model.load_weights(f'{MODEL}_best.hdf6')

In [ ]:
AUGMENTATIONS = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    A.OneOf([
        A.CenterCrop(height=220, width=200),
        A.CenterCrop(height=200, width=220),
    ],p=0.5),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    A.GaussianBlur(p=0.05),
    A.HueSaturationValue(p=0.5),
    A.RGBShift(p=0.5),
    A.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    A.Resize(200, 160)
])

In [ ]:
test_datagen = ImageDataAugmentor( 
    rescale=1./255,
    augment = AUGMENTATIONS,
    validation_split=VAL_SPLIT,
)

In [ ]:
test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=df_submis,
    directory=PATH_to_WORKDIR+'/test_upload',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [ ]:
tta_steps = 10 # берем среднее из 10 предсказаний
predictions = []

for i in range(tta_steps):
    preds = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
    predictions.append(preds)

pred = np.mean(predictions, axis=0)

In [ ]:
predictions = np.argmax(pred, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission_TTA.csv', index=False)


In [ ]:
scores = model.evaluate(test_datagen, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Clean 
import shutil
shutil.rmtree(PATH_to_WORKDIR)